
# Quickstart guide

This example demonstrates how to build a simple content-based audio retrieval model and evaluate the retrieval accuracy on a small song dataset, CAL500. This dataset consists of 502 western pop songs, performed by 499 unique artists. Each song is tagged by at least three people using a standard survey and a fixed tag vocabulary of 174 musical concepts.

This package includes a loading utility for getting and processing this dataset, which makes loading quite easy.

In [1]:
from cbar.datasets import fetch_cal500

X, Y = fetch_cal500()

Calling `fetch_cal500()` initally downloads the CAL500 dataset to a subfolder of your home directory. You can specify a different location using the `data_home` parameter (`fetch_cal500(data_home='path')`). Subsequents calls simply load the dataset.

The raw dataset consists of about 10,000 39-dimensional features vectors
per minute of audio content which were created by

1. Sliding a half-overlapping short-time window of 12 milliseconds over each song's waveform data.
2. Extracting the 13 mel-frequency cepstral coefficients.
3. Appending the instantaneous first-order and second-order derivatives.

Each song is, then, represented by exactly 10,000 randomly subsampled, real-valued feature vectors as a *bag-of-frames*. The *bag-of-frames* features are further processed into one *k*-dimensional feature vector by encoding the feature vectors using a codebook and pooling them into one compact vector.

Specifically, *k*-means is used to cluster all frame vectors into *k* clusters. The resulting cluster centers correspond to the codewords in the codebook. Each frame vector is assigned to its closest cluster center and a song represented as the counts of frames assigned to each of the *k* cluster centers.

By default, `fetch_cal500()` uses a codebook size of 512 but this size is easily modified with the `codebook_size` parameter (`fetch_cal500(codebook_size=1024)`).

In [2]:
X.shape, Y.shape

((502, 512), (502, 174))

Let's split the data into training data and test data, fit the model on the training data, and evaluate it on the test data. Import and instantiate the model first.

In [3]:
from cbar.loreta import LoretaWARP

model = LoretaWARP(n0=0.1, valid_interval=1000)

Then split the data and fit the model using the training data.

In [4]:
from cbar.cross_validation import train_test_split_plus

(X_train, X_test,
 Y_train, Y_test,
 Q_vec, weights) = train_test_split_plus(X, Y)

%time model.fit(X_train, Y_train, Q_vec, X_test, Y_test)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


iter:        0, stepsize:    0.100, P10: 0.168, AP: 0.165, loss: 0.000
iter:     1000, stepsize:    0.100, P10: 0.177, AP: 0.193, loss: 1.000
iter:     2000, stepsize:    0.100, P10: 0.201, AP: 0.198, loss: 0.993
iter:     3000, stepsize:    0.100, P10: 0.195, AP: 0.198, loss: 0.978
iter:     4000, stepsize:    0.100, P10: 0.195, AP: 0.200, loss: 0.965
iter:     5000, stepsize:    0.100, P10: 0.192, AP: 0.198, loss: 0.954
iter:     6000, stepsize:    0.100, P10: 0.193, AP: 0.197, loss: 0.949
iter:     7000, stepsize:    0.100, P10: 0.192, AP: 0.197, loss: 0.933
iter:     8000, stepsize:    0.100, P10: 0.175, AP: 0.195, loss: 0.919
iter:     9000, stepsize:    0.100, P10: 0.187, AP: 0.195, loss: 0.919
iter:    10000, stepsize:    0.100, P10: 0.183, AP: 0.194, loss: 0.896
iter:    11000, stepsize:    0.100, P10: 0.188, AP: 0.196, loss: 0.895
iter:    12000, stepsize:    0.100, P10: 0.189, AP: 0.196, loss: 0.900
iter:    13000, stepsize:    0.100, P10: 0.189, AP: 0.194, loss: 0.894
2019-0

cbar.loreta.LoretaWARP(max_iter=100000, k=30, n0=0.1, n1=0.0, rank_thresh=0.1, lambda_=0.1, loss='warp', max_dips=10, valid_interval=1000)

Now, predict the scores for each query with all songs. Ordering the songs from highest to lowest score corresponds to the ranking.

In [5]:
Y_score = model.predict(Q_vec, X_test)

Evaluate the predictions.

In [6]:
from cbar.evaluation import Evaluator
from cbar.utils import make_relevance_matrix

n_relevant = make_relevance_matrix(Q_vec, Y_train).sum(axis=1)

evaluator = Evaluator()
evaluator.eval(Q_vec, weights, Y_score, Y_test, n_relevant)
evaluator.prec_at

defaultdict(list,
            {1: [0.18292682926829268],
             2: [0.16768292682926828],
             3: [0.1727642276422764],
             4: [0.1717479674796748],
             5: [0.1693089430894309],
             6: [0.1660569105691057],
             7: [0.169904181184669],
             8: [0.17531213704994195],
             9: [0.18318815331010452],
             10: [0.18567799070847857],
             11: [0.19069308063210505],
             12: [0.18736097912927183],
             13: [0.19493084828450685],
             14: [0.19983123245318368],
             15: [0.20638127116785654],
             16: [0.21725246298417034],
             17: [0.22199265195588724],
             18: [0.2300584762161161],
             19: [0.23003708126532452],
             20: [0.2357371898858439]})

## Cross-validation

The `cv` function in the `cross_validation` module offers an easy way to evaluate a retrieval method on multiple splits of the data. Let's run the same experiment on three folds.

In [7]:
from cbar.cross_validation import cv

In [8]:
%time cv('cal500', 512, n_folds=3, method='loreta', n0=0.1, valid_interval=1000)

2019-04-09 04:06:25,780 [MainThread  ] [INFO ]  Running CV with 3 folds ...
2019-04-09 04:06:43,751 [MainThread  ] [INFO ]  Validating fold 0 ...
iter:        0, stepsize:    0.100, P10: 0.160, AP: 0.155, loss: 0.000
iter:     1000, stepsize:    0.100, P10: 0.192, AP: 0.189, loss: 0.999
iter:     2000, stepsize:    0.100, P10: 0.200, AP: 0.190, loss: 0.988
iter:     3000, stepsize:    0.100, P10: 0.190, AP: 0.182, loss: 0.976
iter:     4000, stepsize:    0.100, P10: 0.180, AP: 0.178, loss: 0.959
iter:     5000, stepsize:    0.100, P10: 0.177, AP: 0.177, loss: 0.940
iter:     6000, stepsize:    0.100, P10: 0.175, AP: 0.180, loss: 0.931
iter:     7000, stepsize:    0.100, P10: 0.176, AP: 0.177, loss: 0.918
iter:     8000, stepsize:    0.100, P10: 0.170, AP: 0.178, loss: 0.899
iter:     9000, stepsize:    0.100, P10: 0.177, AP: 0.175, loss: 0.889
iter:    10000, stepsize:    0.100, P10: 0.172, AP: 0.175, loss: 0.878
iter:    11000, stepsize:    0.100, P10: 0.165, AP: 0.174, loss: 0.870
20

The cross-validation results including retrieval method parameters are written to a JSON file. For each dataset three separate result files for mean average precision (MAP), precision-at-*k*, and precision-at-10 as a function of relevant training examples are written to disk. Here are the mean average precision values of the last cross-validation run.

In [9]:
import json
import os
from cbar.settings import RESULTS_DIR

results = json.load(open(os.path.join(RESULTS_DIR, 'cal500_mean_ap.json')))
results[list(results.keys())[-1]]['precision']

[0.17251897309918962, 0.17584143541984665, 0.17753117399618143]

## Start cross-validation with the CLI

This package comes with a simple CLI which makes it easy to start cross-validation experiments from the command line. The CLI enables you to specify a dataset and a retrieval method as well as additional options in one line.

To start an experiment on the CAL500 dataset with the LORETA retrieval method, use the following command.

```
$ cbar crossval --dataset cal500 loreta 
```

This simple command uses all the default parameters for LORETA but you can specify all parameters as arguments to the `loreta` command. To see the available options for the `loreta` command, ask for help like this.

```
$ cbar crossval loreta --help
Usage: cbar crossval loreta [OPTIONS]

Options:
  -n, --max-iter INTEGER        Maximum number of iterations
  -i, --valid-interval INTEGER  Rank of parameter matrix W
  -k INTEGER                    Rank of parameter matrix W
  --n0 FLOAT                    Step size parameter 1
  --n1 FLOAT                    Step size parameter 2
  -t, --rank-thresh FLOAT       Threshold for early stopping
  -l, --lambda FLOAT            Regularization constant
  --loss [warp|auc]             Loss function
  -d, --max-dips INTEGER        Maximum number of dips
  -v, --verbose                 Verbosity
  --help                        Show this message and exit.
```